In [1]:
# import statements

import pandas as pd
import numpy as np
import seaborn as sns
import datetime
from matplotlib import pyplot as plt

In [ ]:
# Read in the 2018 data 
df = pd.read_csv('eda_structuring_with_python_dataset1.csv') 
df.head()

#convert date column to datetime dtype
df['date'] = pd.to_datetime(df['date'])

# Shape check on dataframe
df.shape

#check for duplicates, check by shape
df.drop_duplicates().shape


In [ ]:
# Locations with most strikes in a single day

# Sort by number of strikes in descending order.
df.sort_values(by = 'number_of_strikes', ascending = False).head(10)

# Locations with most days with at least one lightning strike
## Identify the locations that appear most in the dataset.
df.center_point_geom.value_counts()

## Identify the top 20 locations with most days of lightning.
df.center_point_geom.value_counts()[:20].rename_axis('unique_values').reset_index(name = 'counts').style.background_gradient()


# Lightning strikes by day of week
## Create two new columns.
df['week'] = df.date.dt.isocalendar().week
df['weekday'] = df.date.dt.day_name()
df.head()

## Calculate the mean count of lightning strikes for each weekday.
df[['weekday', 'number_of_strikes']].groupby(['weekday']).mean()

## Define order of days for the plot.
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday','Saturday','Sunday']

In [ ]:
# Create boxplots of strike counts for each day of week.
g = sns.boxplot(data = df,
            x = 'weekday',
            y = 'number_of_strikes',
            order = weekday_order,
            showfliers = False
            );
g.set_title('Lightning distribution per weekday (2018)');

In [ ]:
# Monthly lightning strikes 2016-2018

# import 2016-2017 data
df_2 = pd.read_csv('eda_structuring_with_python_dataset2.csv')
df_2.head()

#convert date column to datetime dtype
df_2['date'] = pd.to_datetime(df_2['date'])

# Create a new dataframe combining 2016–2017 data with 2018 data.
union_df = pd.concat([df.drop(['weekday', 'week'], axis = 1), df_2], ignore_index = True)
union_df.head()

#Add 3 new column
union_df['year'] = union_df.date.dt.year
union_df['month']= union_df.date.dt.month
union_df['month_txt'] = union_df.date.dt.month_name()
union_df.head()

In [ ]:
# Calculate total number of strikes per year
union_df[['year', 'number_of_strikes']].groupby.(['year']).sum()

# Calculate total lightning strikes for each month of each year.
lightning_by_month = union_df.groupby(['month_txt', 'year']).agg(
    number_of_strikes = pd.NamedAgg(column = 'number of strikes', aggfunc = sum)
    ).reset_index()
lightning_by_month.head()

# Calculate total lightning strikes for each year.
lightning_by_year = union_df.groupby(['year']).agg(
    year_strikes = pd.NamedAgg(column = 'number_of_strikes', aggfunc=sum)
).reset_index()
lightning_by_year.head()

In [ ]:
# Combine `lightning_by_month` and `lightning_by_year` dataframes into single dataframe.
precentage_lightning = lightning_by_month.merge(lightning_by_year, on = 'year')
precentage_lightning.head()

# Create new `percentage_lightning_per_month` column.
precentage_lightning['precentage_lightning_per_month'] = (percentage_lightning.number_of_strikes/ percentage_lightning.year_strikes * 100.0)
percentage_lightning.head()

In [ ]:
# Bar plot precentages by month

plt.figure(figsize = (10,6));

month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

sns.barplot(
    data = precentage_lightning,
    x = 'month_txt',
    y = 'precentage_lightning_per_month',
    hue = 'year',
    order = month_order);
plt.xlabel("Month");
plt.ylabel("% of Lightning strikes");
plt.title("% of lightning strikes each month (2016 - 2018)");